This notebook a continuation of the previous one. In this notebook: data structures!

### List

* A list of the simplest useful data structure.
* A list is actually a sequence of fixed-size slots, each one containing a pointer to a piece of data elsewhere on the machine.
* Lists are a fixed length. When all of a list's slots are saturated, the list is expanded. The precise expansion algorithm used is implementation-dependent, but in Python the size of the list is generally doubled. Doubling is achieved by allocating as many new slots as necessary to achieve the new length.
* Doubling requires copying the entire list over to a new location. This is necessary to ensure that the list slots are contiguous in memory, and thus to ensure that list access stays a constant speed operation.
* Lists are technically pointer arrays, or they would be if you were doing this in C.
* Lists have the following operation runtimes:
  * $O(1)$ access by index. The index value is multiplied by pointer length (64 bits on a modern machine) to arrive at the correct location in memory.
  * $O(1)$ append. Even though append operations ocassionally trigger list doubling, which requires an $O(n)$ copy, this operation is considered so rare that list append is still $O(1)$ in amortized time.
  * $O(1)$ pop. See append.
  * $O(n)$ insert. Because a list's pointers are contiguous in memory, inserting into a list requires shifting every subsequent pointer over one index.
  * $O(n)$ remove.
  * $O(n)$ access by value if the list is unsorted, otherwise, $O(\log{n})$ access by value.


### Hash map
* A hash map (or, in Python, dictionary) is a key-value pair structure.
* As of Python 3.6, a hash map in Python consists of two lists. Each time a value is inserted into the hash map, it is hashed using a fast equidistributed function. The trailing digits of this hash are used as the lookup index for a sparse list. The contents of the sparse list is indices in a dense list, or None if there is no object in the dense list already which has this value.

  If the sparse list value is None, a (pointer, hash) entry for the value is appended to the end of the dense list, and the value in the sparse list is set to the next index.
  
  If the sparse list value is not None, e.g. there is already a value stored that has this hash, the pointer to that value is modified to point to that value *and* to the new value, with the new value appearing as a subsequent node in a singly linked list of nodes.
* At lookup time, the key is hashed, and the hash is used to index into the sparse array, which is used to index into the dense array. In the dense array, the full hash of the leading value struct is compared with the full hash of the key. If they match, the value is returned. If they do not match, the next value in the linked list is selected. This operation continues until a matching value is found.
* The larger the number of items and the smaller the hash edge used for the key lookups, the larger the number of key collisions. To avoid key collisions, Python attempts to keep at least 1/3 of the hash map empty. When this threshold is exceed the hash map is expanded: another digit of the hash is consumed, the sparse list is resized, etcetera.
* Key values are restricted to those values which are immutable, and hence, hashable. E.g. a notable exception is lists.
* Hash maps have the following operation runtimes:
  * $O(1)$ lookup by key. Key collisions are rare, so the amortized time is constant.
  * $O(1)$ insert. Key collisions and resize operations are rare, so the amortized time is constant.
  * $O(1)$ remove.


### Set
* A set is an iterable containing only unique values. Just like the mathematical definition.
* Sets provide a fast uniqueness check. Under the hood they are just hash maps.
* Runtimes:
  * $O(1)$ insert.
  * $O(1)$ remove.
  * $O(1)$ set membership check.
  * $O(n)$ search.


### Singly linked list
* A singly-linked list is a set of nodes with pointers in between nodes. Pointers only point one way, away from a lead node.
* This is a very simple data structure. No fanciness at all. They're good for access to first and last.
* Singly-linked lists have the following operation runtimes:
  * $O(1)$ prepend and append, so long as you retain a pointer to the last element.
  * $O(1)$ peak. The first element is always easy to get.
  * $O(n)$ lookup and insert. Looking up a value or index requires jumping through pointers to get to that value or index.


### Binary search tree
* A binary search tree is a tree where each node has up to two children. The rule being that the left subchild is always smaller than the parent value, and the right subchild, bigger.
* The value of this data structure is that it allows for faster search, at the cost of slower inserts and deletes.
* The runtimes are:
  * $O(\log{n})$ search.
  * $O(\log{n})$ insert.
  * $O(\log{n})$ delete.


### Stack
* A first-in, last-out (FILO) data structure. Easily implemented using a list or singly linked list.
* This data structure has practical application in programming language design: functional state in most programming languages is managed via stack frames on a call stack.
* Runtimes are:
  * $O(1)$ pop (get last).
  * $O(1)$ insert.
  * $O(n)$ lookup.


### Queue
* A first-in, first-out (FIFO) data structure.
* A queue can be implemented using a singly-linked list (see the definition below).
* Runtime characteristics are:
  * $O(1)$ enqueue.
  * $O(1)$ dequeue (get first).
  * $O(n)$ lookup.

In [22]:
class Node:
    def __init__(self, v):
        self.v = v
        self.next = None
    
class Q:
    def __init__(self):
        self.first = None
        self.last = None
    
    def enqueue(self, v):
        if self.first is None:
            self.first = self.last = Node(v)
        else:
            _next = Node(v)
            self.last.next = _next
            self.last = self.last.next
    
    def dequeue(self):
        first = self.first
        self.first = self.first.next
        return first.v

In [23]:
q = Q()
q.enqueue(1)
q.enqueue(2)
q.enqueue(3)
print(q.dequeue())
print(q.dequeue())
print(q.dequeue())

1
2
3


### Dequeue
* A dequeue is a double-sided queue. You can consume both first and last values of a dequeue efficiently.
* A dequeue is implemented using a double-sided list.
* Runtime characteristics are:
  * $O(1)$ enqueue and push.
  * $O(1)$ dequeue and pop.
  * $O(n)$ lookup.

In [18]:
class Node:
    def __init__(self, v):
        self.v = v
        self.next = None
        self.prev = None
    
class Dequeue:
    def __init__(self):
        self.first = None
        self.last = None
    
    def push(self, v):
        if self.first is None:
            self.first = self.last = Node(v)
        else:
            first = Node(v)
            first.next = self.first
            self.first.prev = first
            self.first = first
    
    def pop(self):
        first = self.first
        self.first = self.first.next
        self.first.prev = None
        return first.v
    
    def enqueue(self, v):
        if self.last is None:
            self.first = self.last = Node(v)
        else:
            last = Node(v)
            last.prev = self.last
            self.last.next = last
            self.last = last
    
    def dequeue(self):
        last = self.last
        self.last.prev.next = None
        self.last = self.last.prev
        return last.v

In [19]:
dq = Dequeue()
dq.enqueue(1)
dq.enqueue(2)
dq.enqueue(3)
print(dq.dequeue())
print(dq.pop())

3
1


### Binary heap
* A **binary heap** (or just **heap**) is a binary tree whose nodes always have an either greater-than or lesser-than relationship with their parents. If the root node is the smallest value in the tree, the tree is a min heap; if the root node is the largest value in the tree, it is a max heap.
* Heaps provide a $O(1)$ access to the minimum or maximum element of the tree. For this reason they are the data structure of choice for the implementation of **priority queues**.
* An advanced variant known as a **Fibbonacci heap** reduces this runtime further, down to $O(1)$. This is the "killer feature" of heaps in practical applications, but is not relevant to the simple binary tree implementation.
* Binary heaps are implemented on disk as a layered array with progressively larger node counts. E.g. the first element of the array is a pointer to the top array element. The next two indices are its children. The next four are its children. And so on. Note: binary trees are implemented on disk the same way.
* Runtimes are:
  * $O(\log{n})$ insert.
  * $O(1)$ get-min or get-max.
  * $O(\log{n})$ get and delete.

### Balanced binary search trees (AVL tree)
* A balanced binary search tree is one in which the height of the tree is close to optimal for the given number of nodes.
* The first algorithmic implementation achieving this effect is the AVL tree algorithm. AVL trees work by rebalancing the tree using node pivot operations whenever a node insertion would violate the height constraint to rebalance the tree.
* Runtimes are the same as that of an ordinary binary search tree.